In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import iaaft
import xarray as xr
import math

ModuleNotFoundError: No module named 'iaaft'

In [ ]:
data = Dataset("CMIP6_8.5_CESM2_regridded.nc", 'r')

# print variables 
for var in data.variables:
    print(var)

lat = data.variables['lat']
lon = data.variables['lon']
time = data.variables['time']
#realization = data.variables['realization'][:]
tp = data.variables['tas']

print(lat)
print(lon)
print(time)
#print(realization)
print(tp)